In [1]:
from typing import TypedDict

class SupportState(TypedDict):
    query: str
    intent: str
    observation: str
    response: str


In [2]:
def get_order_status(order_id: int):
    return f"Order {order_id} is out for delivery."

def raise_ticket(issue: str):
    return f"Support ticket created for issue: {issue}"


In [3]:
from langchain_community.chat_models import ChatOllama

llm = ChatOllama(model="llama3")


C:\Users\apoliset\AppData\Local\Temp\ipykernel_30448\2342230918.py:3: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the `langchain-ollama package and should be used instead. To use it run `pip install -U `langchain-ollama` and import as `from `langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3")


In [4]:
def intent_node(state: SupportState):
    prompt = f"""
Classify the customer query into ONE of the following intents:
- faq
- order_status
- escalation

Query: {state['query']}

Respond ONLY with the intent name.
"""
    intent = llm.invoke(prompt).content.strip().lower()
    return {"intent": intent}


In [5]:
def action_node(state: SupportState):
    intent = state["intent"]
    query = state["query"]

    if intent == "order_status":
        order_id = int("".join(filter(str.isdigit, query)))
        return {"observation": get_order_status(order_id)}

    if intent == "escalation":
        return {"observation": raise_ticket(query)}

    return {"observation": "FAQ handled"}


In [6]:
def response_node(state: SupportState):
    prompt = f"""
Customer Query: {state['query']}
System Result: {state['observation']}

Generate a polite customer support response.
"""
    response = llm.invoke(prompt).content
    return {"response": response}


In [7]:
from langgraph.graph import StateGraph

graph = StateGraph(SupportState)

graph.add_node("intent", intent_node)
graph.add_node("action", action_node)
graph.add_node("response", response_node)

graph.set_entry_point("intent")
graph.add_edge("intent", "action")
graph.add_edge("action", "response")

support_bot = graph.compile()


In [8]:
support_bot.invoke({
    "query": "What is your return policy?"
})


{'query': 'What is your return policy?',
 'intent': 'faq',
 'observation': 'FAQ handled',
 'response': "Thank you for reaching out to us about our return policy! We're happy to help.\n\nAt [Your Company Name], we want to ensure that you're completely satisfied with your purchase. If, for any reason, you're not satisfied, we offer a flexible return policy.\n\nHere are the details:\n\n* You can return or exchange an item within [timeframe, e.g., 30 days] of delivery.\n* Items must be in their original condition with all tags and packaging intact.\n* To initiate a return or exchange, please contact our customer service team at [phone number/email address]. We'll provide you with a Return Merchandise Authorization (RMA) number and guide you through the process.\n\nPlease note that some items may have specific return conditions or restrictions. In these cases, we'll let you know upfront so you can make an informed decision.\n\nWe're committed to providing excellent customer service and want

In [9]:
support_bot.invoke({
    "query": "Check order status for order 456"
})


{'query': 'Check order status for order 456',
 'intent': 'order_status',
 'observation': 'Order 456 is out for delivery.',
 'response': 'Here\'s a polite customer support response:\n\n"Hello there! I\'m happy to help you with your query. According to our records, your order number 456 is currently out for delivery. This means that it has been handed over to the courier service and should arrive at your doorstep shortly.\n\nIf you have any specific questions about the delivery timeline or would like an estimated arrival time, please feel free to ask and I\'ll do my best to assist you.\n\nThanks for shopping with us! If there\'s anything else I can help with, please don\'t hesitate to reach out."'}

In [10]:
support_bot.invoke({
    "query": "My payment failed and money was deducted"
})


{'query': 'My payment failed and money was deducted',
 'intent': 'escalation',
 'observation': 'Support ticket created for issue: My payment failed and money was deducted',
 'response': "Here's a polite customer support response:\n\n**Subject: Re: Payment Issue - Assistance Requested**\n\nDear [Customer Name],\n\nThank you for reaching out to us about the issue with your recent payment. We apologize for any inconvenience this has caused and appreciate you bringing it to our attention.\n\nWe're here to help resolve the problem as quickly as possible. Our team will investigate why the payment failed and work on reversing the deduction from your account, if necessary.\n\nIn the meantime, can you please confirm some details about your transaction? Specifically:\n\n1. The order number or reference code associated with the payment\n2. The amount that was deducted from your account\n\nThis information will help us to expedite our investigation and provide a solution for you.\n\nWe value your 